# Hidden Markov model
 
Finding the probability of observing a particular state as the next state in the HMM involves a matrix-vector multiplication, where the matrix contains the state transition probabilities. Depending on the sparsity level of this matrix, some loops must be unrolled automatically (fully or partially), and the transformations $0 ∗ x = 0$ and $0 + x = x$ need to be performed.

Kenichi Asai posted a simplified version of the HMM problem. Given a general matrix-vector multiplication program that is used to compute the next state in HMM:

<https://github.com/StagedHPC/shonan-challenge/tree/master/problems/hmm>

A desired optimization is to unfold the loop for each row of a particular adjacency matrix if the number of elements of the row is below a certain threshold value.

The original program in C:


```c
int ∗hmm(int n, int ∗∗a, int ∗v) {
  int ∗w = (int∗) calloc(n, sizeof (int)),
  for (int i = 0, i < n; i++)
    for (int j =0; j < n; j++)
      w[i] += a[i][j] ∗ v[j];
  return w;
}
```

In [1]:
function hmm(a::Matrix{T}, v::Vector{T}) where {T}
    n = size(a, 1)
    @assert size(a, 2) == n
    @assert length(v) == n

    w = zeros(T, size(v))
    for i in 1:n, j in 1:n
        w[i] += a[i, j] * v[j]
    end
    return w
end

hmm (generic function with 1 method)

In [2]:
using Test, CompTime

In [3]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [4]:
A = [11 12 13; 21 22 23; 31 32 33]

3×3 Matrix{Int64}:
 11  12  13
 21  22  23
 31  32  33

In [5]:
V3 = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [6]:
expected3 = [74, 134, 194]

3-element Vector{Int64}:
  74
 134
 194

In [7]:
@test hmm(A, V3) == expected3

Test Passed

First of all, let's replace the mutable array `a` with an immutable tuple of tuples.

In [8]:
A3 = ((11, 12, 13), (21, 22, 23), (31, 32, 33))

((11, 12, 13), (21, 22, 23), (31, 32, 33))

In [9]:
function hmm(a::NTuple{N, NTuple{N, T}}, v::Vector{T}) where {N, T}
    @assert length(v) == N

    w = zeros(T, N)
    for i in 1:N, j in 1:N
        w[i] += a[i][j] * v[j]
    end
    return w
end

hmm (generic function with 2 methods)

In [10]:
@test hmm(A3, V3) == expected3

Test Passed

In [11]:
function hmm_rt(a, v::Vector{T}) where {T}
    N = length(a)
    @assert length(v) == N

    w = zeros(T, N)
    for i in 1:N
        ai = a[i]
        @assert length(ai) == N
        for j in 1:N
            w[i] += ai[j] * v[j]
        end
    end
    return w
end

hmm_rt (generic function with 1 method)

In [12]:
@test hmm_rt(A3, V3) == expected3

Test Passed

In [13]:
@ct_enable function hmm_ct1(@ct(a), v::Vector{T}) where {T}
    @ct(N = length(a))
    @assert length(v) == @ct(N)

    w = zeros(T, @ct(N))
    for i in 1:@ct(N)
        ai = a[i]
        @assert length(ai) == @ct(N)
        for j in 1:@ct(N)
            w[i] += ai[j] * v[j]
        end
    end
    return w
end

runtime (generic function with 1 method)

In [14]:
debug(hmm_ct1, Val{A3}, V3) |> cleanup

quote
    @assert length(v) == 3
    w = zeros(T, 3)
    for i = 1:3
        ai = a[i]
        @assert length(ai) == 3
        for j = 1:3
            w[i] += ai[j] * v[j]
        end
    end
    return w
end

In [15]:
@test hmm_ct1(Val{A3}, V3) == expected3

Test Passed

In [16]:
@ct_enable function hmm_ct2(@ct(a), v::Vector{T}) where {T}
    @ct(N = length(a))
    @assert length(v) == @ct(N)

    w = zeros(T, @ct(N))
    @ct_ctrl for i in 1:N
        ai = a[@ct(i)]
        @assert length(ai) == @ct(N)
        @ct_ctrl for j in 1:N
            w[@ct(i)] += ai[@ct(j)] * v[@ct(j)]
        end
    end
    return w
end

runtime (generic function with 2 methods)

In [17]:
debug(hmm_ct2, Val{A3}, V3) |> cleanup

quote
    @assert length(v) == 3
    w = zeros(T, 3)
    ai = a[1]
    @assert length(ai) == 3
    w[1] += ai[1] * v[1]
    w[1] += ai[2] * v[2]
    w[1] += ai[3] * v[3]
    ai = a[2]
    @assert length(ai) == 3
    w[2] += ai[1] * v[1]
    w[2] += ai[2] * v[2]
    w[2] += ai[3] * v[3]
    ai = a[3]
    @assert length(ai) == 3
    w[3] += ai[1] * v[1]
    w[3] += ai[2] * v[2]
    w[3] += ai[3] * v[3]
    return w
end

In [18]:
@test hmm_ct2(Val{A3}, V3) == expected3

Test Passed

In [19]:
@ct_enable function hmm_ct3(@ct(a), v::Vector{T}) where {T}
    @ct(N = length(a))
    @assert length(v) == @ct(N)

    w = zeros(T, @ct(N))
    @ct_ctrl for i in 1:N
        @ct(ai = a[i])
        @ct(@assert length(ai) == N)
        @ct_ctrl for j in 1:N
            w[@ct(i)] += @ct(ai[j]) * v[@ct(j)]
        end
    end
    return w
end

runtime (generic function with 3 methods)

In [20]:
debug(hmm_ct3, Val{A3}, V3) |> cleanup

quote
    @assert length(v) == 3
    w = zeros(T, 3)
    w[1] += 11 * v[1]
    w[1] += 12 * v[2]
    w[1] += 13 * v[3]
    w[2] += 21 * v[1]
    w[2] += 22 * v[2]
    w[2] += 23 * v[3]
    w[3] += 31 * v[1]
    w[3] += 32 * v[2]
    w[3] += 33 * v[3]
    return w
end

In [21]:
@test hmm_ct3(Val{A3}, V3) == expected3

Test Passed

Now we can implement some optimizations.

- Eliminate multiplication by `zero(T)` and `one(T)`.
- Unroll the inner loop only is `length(ai) < threshold`.

In [22]:
A5 =
   (2, 0, 0, 1, 0),
   (0, 0, 1, 0, 0),
   (0, 1, 0, 0, 0),
   (0, 0, 4, 5, 6),
   (0, 0, 1, 0, 1)

((2, 0, 0, 1, 0), (0, 0, 1, 0, 0), (0, 1, 0, 0, 0), (0, 0, 4, 5, 6), (0, 0, 1, 0, 1))

In [23]:
V5 = [1, 2, 3, 4, 5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [24]:
debug(hmm_ct3, Val{A5}, V5) |> cleanup

quote
    @assert length(v) == 5
    w = zeros(T, 5)
    w[1] += 2 * v[1]
    w[1] += 0 * v[2]
    w[1] += 0 * v[3]
    w[1] += 1 * v[4]
    w[1] += 0 * v[5]
    w[2] += 0 * v[1]
    w[2] += 0 * v[2]
    w[2] += 1 * v[3]
    w[2] += 0 * v[4]
    w[2] += 0 * v[5]
    w[3] += 0 * v[1]
    w[3] += 1 * v[2]
    w[3] += 0 * v[3]
    w[3] += 0 * v[4]
    w[3] += 0 * v[5]
    w[4] += 0 * v[1]
    w[4] += 0 * v[2]
    w[4] += 4 * v[3]
    w[4] += 5 * v[4]
    w[4] += 6 * v[5]
    w[5] += 0 * v[1]
    w[5] += 0 * v[2]
    w[5] += 1 * v[3]
    w[5] += 0 * v[4]
    w[5] += 1 * v[5]
    return w
end

In [25]:
expected5 = [6, 3, 2, 62, 8]

5-element Vector{Int64}:
  6
  3
  2
 62
  8

In [26]:
@test hmm_ct3(Val{A5}, V5) == expected5

Test Passed

In [27]:
@ct_enable function hmm_ct_opt(@ct(a), v::Vector{T}) where {T}
    @ct(N = length(a))
    @assert length(v) == @ct(N)

    w = zeros(T, @ct(N))
    @ct_ctrl for i in 1:N
        @ct(ai = a[i])
        @ct(@assert length(ai) == N)
        @ct_ctrl if sum(ai .!= 0) >= 3
            for j in 1:@ct(N)
                w[@ct(i)] += @ct(a[i])[j] * v[j]
            end
        else
            @ct_ctrl for j in 1:N
                @ct_ctrl if ai[j] == one(T)
                    w[@ct(i)] += v[@ct(j)]
                elseif ai[j] != zero(T)
                    w[@ct(i)] += @ct(ai[j]) * v[@ct(j)]
                end
            end
        end
    end
    return w
end

runtime (generic function with 4 methods)

In [28]:
debug(hmm_ct_opt, Val{A5}, V5) |> cleanup

quote
    @assert length(v) == 5
    w = zeros(T, 5)
    w[1] += 2 * v[1]
    w[1] += v[4]
    w[2] += v[3]
    w[3] += v[2]
    for j = 1:5
        w[4] += ((0, 0, 4, 5, 6))[j] * v[j]
    end
    w[5] += v[3]
    w[5] += v[5]
    return w
end

In [29]:
@test hmm_ct_opt(Val{A5}, V5) == expected5

Test Passed